[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MatteoFalcioni/Learning-LangGraph/blob/main/notebooks/7.5_recap.ipynb)

### Setup

#### Install requirements

In [ ]:
%pip install -q -U -r https://raw.githubusercontent.com/MatteoFalcioni/Learning-LangGraph/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### local (notebooks or files)

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # load api keys

True

#### Colab

In [ ]:
from google.colab import userdata
import os

REQUIRED_KEYS = [
    'OPENAI_API_KEY',
    'LANGSMITH_TRACING',
    'LANGSMITH_ENDPOINT',
    'LANGSMITH_API_KEY',
    'LANGSMITH_PROJECT'
]

def _set_colab_keys(key : str):
    # Retrieve the secret value using its key/name
    secret_value = userdata.get(key)
    # set it as a standard OS environment variable
    os.environ[key] = secret_value

for key in REQUIRED_KEYS:
    _set_colab_keys(key)

# Recap (lessons 4 to 7)

In the last few lessons we saw the "intermediate level" core concepts of LangGraph: let's recap them.

## Reducers

They specify how to update state variables + they fix the problem of parallel access from different nodes to the same state key.

You need to annotate the state var with your reducer in order for LangGraph to be aware of the reducer. 

You should always specify a reducer. The only time you may not specify one is when working with very simple types like strings or ints/floats. But LangGraph will handle these by overwriting the state var every time.

Remember the `(left, right) -> left + right` philosophy.

### Example

```python
# define the reducer
def reduce_list(left: list | None, right: list | None) -> list:
    if not left:
        left = []
    if not right:
        right = []

    updated_list = left + right
    return updated_list


# use it in state:
from typing import Annotated

class ReducedState(TypedDict):
    foo: Annotated[list[int], reduce_list]   # annotate the foo key with the reducer
```

## `Command`

Is a type that can be used:

### In nodes

To combine control flows and state update: remember to specify the possible nodes as `Command[Literal<node name>]`. Can be more than one node.

#### Example 

```python
def update_counter_node(state: MyState) -> Command[Literal["accepted_user_node", "rejected_user_node"]]:

    # this does the work of a conditional edge
    user_id = state.get('user_id', '')  # check state
    
    # decide next node
    if user_id == 'Matteo':
        goto='accepted_user_node'
    else:
        goto='rejected_user_node'

    return Command(
        # state update
        update={"counter": +1},  # again, dictionary of updates
        # control flow
        goto=goto   
    )
    
```

#### In tools

To update the graph state from tools. 

Remember to:

- always include a tool message in the updates if using Command in tools

- use the correct tool call id by accessing it with `runtime`

- use `runtime.state` if you want to access state in the tool (do not pass `state : Mystate`)

##### Example

```python
@tool
def get_user_info(runtime: ToolRuntime) -> str:
    """Look up user info."""
    user_id = runtime.state["user_id"]  # runtime to access state

    result_string = "User is Matteo" if user_id == "user_123" else "Unknown user"

    # updates to state 
    return Command(
        update = {
            "messages" : [  # must be a list to update messages
                ToolMessage(  # always needed
                    content=result_string, 
                    tool_call_id=runtime.tool_call_id  
                )
            ]
        }
    )

## Memory

We saw that we can give memory to our agents just by compiling the graph with a checkpointer. 

```python

builder = StateGraph(MyState)
builder.add_node("agent_node", agent_node)
builder.add_edge(START, "agent_node")

checkpointer = InMemorySaver()

graph = builder.compile(checkpointer=checkpointer)

```

Remember you need to invoke with a config specifying a thread id:

```python 

from langchain_core.messages import HumanMessage

init_state = {"messages" : [HumanMessage(content="Get user info")], "user_id": "user_123"}
config = {"configurable": {"thread_id": "test_123"}}

result = graph.invoke(init_state, config=config)

```

---

## Solutions

### Ex. 4.1

Create a state that stores, other than messages, the key 'report'. This state variable must be a dictionary containing two keys: 'title' and 'content'. You must create a custom reducer as well.   
    *hint: for the reducer, leverage pythonic syntax for merging dicts: `{**left, **right}`

### (My) Solution


In [1]:
from langchain.agents import AgentState
from typing import Annotated

def merge_dicts(left: dict | None, right: dict | None) -> dict:
    """
    Reducer to merge two dictionaries.

    The right dict overwrites the left one if they have overlapping keys.
    This is because when a new report is created, the old one is overwritten (only one report in state at a time)
    """
    if left is None:
        left = {}
    if right is None:
        right = {}
    return {**left, **right}

class MyState(AgentState):
    report: Annotated[dict, merge_dicts]

### Ex. 5.1

Create a node that checks a meaningful state variable (for example, a counter, a username, etc.) and based on the result routes to either the 'accepted_path' node or 'rejected_path' node. You must use `Command(goto=<node name>)` to route.

### (My) Solution


In [ ]:
from langchain.agents import AgentState
from langgraph.types import Command
from typing import Annotated, Literal

def add_int(left: int | None, right: int | None) -> int:
    if left is None:
        left = 0
    if right is None:
        right = 0
    return left + right

class MyState(AgentState):
    counter: Annotated[int, add_int]

def update_counter_node(state: MyState) -> Command[Literal["accepted_user_node", "rejected_user_node"]]:

    counter = state.get('counter', '')  # check state
    
    # decide next node
    if counter > 5:
        goto='accepted_user_node'
    else:
        goto='rejected_user_node'

    return Command(
        # state update
        update={"counter": +1},  # again, dictionary of updates
        # control flow
        goto=goto   
    )

### Ex. 5.2

Create a tool that leverages `Command` to make state updates. Use `runtime` to access state and the correct `tool_call_id`. 

### (My) Solution

In [ ]:
from langchain.tools import tool, ToolRuntime
from langchain_core.messages import ToolMessage
from langgraph.types import Command
from typing import Annotated

@tool 
def write_report(content: Annotated[str, "The content of the report"], runtime: ToolRuntime) -> Command:
    """
    Use this tool to write a report
    """
    state = runtime.state
    topic = state.get("topic", "")

    if topic == "":
        return Command(
            update = {"messages" : [ToolMessage(content="No topic provided, something went wrong", tool_call_id=runtime.tool_call_id)]}
        )

    else:

        output_file = f"reports/report_{topic}.txt"

        with open(output_file, "w") as f:
            f.write(content)

        return Command(
            update = {"messages" : [ToolMessage(content=f"Wrote report on {topic} and saved it to reports/report_{topic}.txt", tool_call_id=runtime.tool_call_id)]}
        )
    

    